In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data from the UN FAO DAD-IS DB
# Domestic Animal Diversity Information System (DAD-IS)
dad_is_df = pd.read_csv('../data/010522_FAO_DAD-IS_export.csv')
dad_is_df = dad_is_df.replace({'Duck (domestic)': 'Duck', 'Duck(domestic) Muscovy duck': 'Muscovy Duck',
                               'Goose (domestic)': 'Goose', 'Dromedary Bactrian Camel': 'Bactrian Camel',
                               'Vicuña': 'Vicuna', 'Yak (domestic)': 'Yak'})

cols_of_interest = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females',
                    'Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females',
                    'Age first parturition (month) AVG', 'Age first parturition (month) MIN', 'Age first parturition (month) MAX',
                    'Parturition interval (day) AVG', 'Parturition interval (day) MIN', 'Parturition interval (day) MAX',
                    'Length productive life (years)', 'Daily gain', 'Carcass weight (kg)', 'Dressing percentage (%)',
                    'Litter size AVG', 'Litter size MIN', 'Litter size MAX']

# Renaming map for the columns to include units where absent.
# Units gleaned from GUI interface here:
# https://www.fao.org/dad-is/browse-by-country-and-species/en/
colname_map = dict((k,k) for k in cols_of_interest)
kg_cols = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females']
for c in kg_cols:
    colname_map[c] = c + ' (kg)'
month_cols = ['Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females']
for c in month_cols:
    colname_map[c] = c + ' (month)'
colname_map['Daily gain'] = 'Daily gain (g)'

# Zero values are non-sensical for all the columns - should be NaN for log-scale plotting
for c in cols_of_interest:
    mask = dad_is_df[c] == 0
    dad_is_df.loc[mask, c] = np.nan
    
# Drop everything that is missing data in all columns of interest. 
dad_is_df = dad_is_df.dropna(axis=0, how='all', subset=cols_of_interest)

#dad_is_df.columns.tolist()
colname_map

/Users/flamholz/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


{'Weight males': 'Weight males (kg)',
 'Weight females': 'Weight females (kg)',
 'Birth weight males': 'Birth weight males (kg)',
 'Birth weight females': 'Birth weight females (kg)',
 'Age maturity males': 'Age maturity males (month)',
 'Age maturity females': 'Age maturity females (month)',
 'Age breeding males': 'Age breeding males (month)',
 'Age breeding females': 'Age breeding females (month)',
 'Age first parturition (month) AVG': 'Age first parturition (month) AVG',
 'Age first parturition (month) MIN': 'Age first parturition (month) MIN',
 'Age first parturition (month) MAX': 'Age first parturition (month) MAX',
 'Parturition interval (day) AVG': 'Parturition interval (day) AVG',
 'Parturition interval (day) MIN': 'Parturition interval (day) MIN',
 'Parturition interval (day) MAX': 'Parturition interval (day) MAX',
 'Length productive life (years)': 'Length productive life (years)',
 'Daily gain': 'Daily gain (g)',
 'Carcass weight (kg)': 'Carcass weight (kg)',
 'Dressing perc

The next few cells are printing out some outlier data to document some clear errors in DAD-IS. 

In [3]:
# Look at some of the absurd litter sizes in the dataset - several > 100!
# Hiding the pigs since commercial pigs can have litters > 10. 
mask = np.logical_and(dad_is_df['Litter size AVG'] > 10,
                      dad_is_df.Specie != 'Pig')
cols =  ['Specie', 'Country', 'Breed/Most common name', 'Parturition interval (day) MIN',
         'Parturition interval (day) AVG', 'Parturition interval (day) MAX',
         'Litter size AVG', 'Litter size MIN', 'Litter size MAX', 
         'Age first parturition (month) AVG',
         'Age first parturition (month) MIN',
         'Age first parturition (month) MAX',
         'Weight males', 'Birth weight males',
         'Weight females', 'Birth weight females']
dad_is_df.loc[mask, cols]

,Specie,Country,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Age first parturition (month) AVG,Age first parturition (month) MIN,Age first parturition (month) MAX,Weight males,Birth weight males,Weight females,Birth weight females
6805,Goat,Croatia,Boer goat,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9662,Goat,North Macedonia,Macedonian Goat,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,35.0,2.50,45.0,2.20
10912,Goat,Oman,Aljabal Alakhdar,NaN,149.0,NaN,128.0,125.0,130.0,24.0,20.0,26.0,36.0,3.34,36.0,3.05
10916,Goat,Oman,Jabbali,148.0,152.0,156.0,126.0,120.0,133.0,23.0,20.0,26.0,36.0,3.30,31.0,2.90
13560,Sheep,Tunisia,Barbarine,NaN,NaN,NaN,112.0,NaN,NaN,24.0,17.0,24.0,NaN,3.50,NaN,3.20
13562,Sheep,Tunisia,Noire de Thibar,NaN,NaN,NaN,130.0,130.0,160.0,20.0,20.0,24.0,NaN,4.32,NaN,NaN
13564,Sheep,Tunisia,Sicilo-Sarde,NaN,NaN,NaN,144.0,NaN,NaN,NaN,20.0,24.0,70.0,3.40,45.0,3.20


In [4]:
# Look at some of the absurd litter sizes in the dataset - several > 100!
# Hiding the pigs since commercial pigs can have litters > 10. 
mask = np.logical_and(dad_is_df['Litter size AVG'] > 10,
                      dad_is_df.Specie != 'Pig')
dad_is_df.loc[mask, cols]

,Specie,Country,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Age first parturition (month) AVG,Age first parturition (month) MIN,Age first parturition (month) MAX,Weight males,Birth weight males,Weight females,Birth weight females
6805,Goat,Croatia,Boer goat,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9662,Goat,North Macedonia,Macedonian Goat,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,35.0,2.50,45.0,2.20
10912,Goat,Oman,Aljabal Alakhdar,NaN,149.0,NaN,128.0,125.0,130.0,24.0,20.0,26.0,36.0,3.34,36.0,3.05
10916,Goat,Oman,Jabbali,148.0,152.0,156.0,126.0,120.0,133.0,23.0,20.0,26.0,36.0,3.30,31.0,2.90
13560,Sheep,Tunisia,Barbarine,NaN,NaN,NaN,112.0,NaN,NaN,24.0,17.0,24.0,NaN,3.50,NaN,3.20
13562,Sheep,Tunisia,Noire de Thibar,NaN,NaN,NaN,130.0,130.0,160.0,20.0,20.0,24.0,NaN,4.32,NaN,NaN
13564,Sheep,Tunisia,Sicilo-Sarde,NaN,NaN,NaN,144.0,NaN,NaN,NaN,20.0,24.0,70.0,3.40,45.0,3.20


In [5]:
# Cattle with have ages at first birth that are very long, e.g. > 10 years.
mask = np.logical_and(dad_is_df['Age first parturition (month) AVG'] > 10*365/12,
                      dad_is_df.Specie == 'Cattle')
dad_is_df.loc[mask, cols]

,Specie,Country,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Age first parturition (month) AVG,Age first parturition (month) MIN,Age first parturition (month) MAX,Weight males,Birth weight males,Weight females,Birth weight females
9935,Cattle,Mozambique,Africander,NaN,421.0,NaN,NaN,NaN,NaN,1266.0,NaN,NaN,NaN,NaN,450.0,NaN


In [6]:
# Look at some absurd birth weights that well-exceed the listed adult weights.
mask = np.logical_or(dad_is_df['Birth weight males'] > 0.9*dad_is_df['Weight males'],
                     dad_is_df['Birth weight females'] > 0.9*dad_is_df['Weight females'])

dad_is_df.loc[mask, cols]

,Specie,Country,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Age first parturition (month) AVG,Age first parturition (month) MIN,Age first parturition (month) MAX,Weight males,Birth weight males,Weight females,Birth weight females
1398,Sheep,Bosnia and Herzegovina,Privorska,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,52.0,46.00,40.0,44.00
2399,Cattle,China,Tibetan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215.0,215.80,197.0,197.70
7322,Rabbit,Indonesia,Rexsi Agrinak,14.0,14.0,16.0,NaN,7.0,8.0,NaN,NaN,NaN,2.9,49.94,2.7,55.27
8894,Chicken,Libya,ALARABY CHICKENS,NaN,NaN,NaN,NaN,NaN,NaN,6.6,6.3,6.8,1.9,1.90,1.5,1.50
11279,Chicken,Philippines,Banaba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,29.86,1.5,29.86
13394,Chicken,Thailand,Khiao Hu0ai Sai chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.50,2.5,2.50
14032,Rabbit,Ukraine,Californian,NaN,NaN,NaN,NaN,8.0,10.0,NaN,NaN,NaN,4.4,450.00,4.4,NaN


In [7]:
# Look at some absurd partruition intervals (inter-birth intervals) that are too short to be reasonable. 
# ... From the website the parturition interval appears to be reported in days. 
# Even for a rabbit this is too short...
mask = dad_is_df['Parturition interval (day) AVG'] < 20 
masked_df = dad_is_df.loc[mask, cols]
for gidx, gdf in masked_df.groupby('Specie'):
    print(gidx)
    for idx, row in gdf.iterrows():
        print(row['Breed/Most common name'])
    
masked_df

Cattle
Blanc-Bleu type mixte
Bali
Donggala
Pesisir (sum )
Sumbawa
Puerto Rican
Winam
Chicken
Mongolian Local Hen
Goat
Criolla del Sur de Mendoza
Creole
Pig
Kalinga
Quezon
Rabbit
Rexsi Agrinak
Sheep
Black Head Somali
Spelsau
Namaqua Afrikaner


,Specie,Country,Breed/Most common name,Parturition interval (day) MIN,Parturition interval (day) AVG,Parturition interval (day) MAX,Litter size AVG,Litter size MIN,Litter size MAX,Age first parturition (month) AVG,Age first parturition (month) MIN,Age first parturition (month) MAX,Weight males,Birth weight males,Weight females,Birth weight females
205,Goat,Argentina,Criolla del Sur de Mendoza,14.0,12.00,10.0,1.30,1.0,3.0,18.00,12.0,22.0,79.6,2.78,46.79,2.54
892,Cattle,Belgium,Blanc-Bleu type mixte,10.0,13.00,18.0,NaN,NaN,NaN,32.00,22.0,38.0,1100.0,46.00,700.00,42.00
5088,Sheep,Ethiopia,Black Head Somali,NaN,10.46,NaN,NaN,NaN,NaN,23.56,NaN,NaN,29.5,NaN,25.80,NaN
6885,Goat,Haiti,Creole,NaN,13.00,NaN,1.75,NaN,NaN,11.00,NaN,NaN,NaN,NaN,NaN,NaN
7149,Cattle,Indonesia,Bali,13.0,14.50,17.0,NaN,1.0,3.0,22.00,26.0,36.0,475.0,18.00,250.00,20.00
7153,Cattle,Indonesia,Donggala,15.0,18.00,24.0,NaN,1.0,1.0,28.00,24.0,33.0,266.0,NaN,266.00,NaN
7164,Cattle,Indonesia,Pesisir (sum ),330.0,12.00,390.0,NaN,NaN,NaN,24.00,21.0,30.0,200.0,13.00,150.00,11.00
7170,Cattle,Indonesia,Sumbawa,12.0,14.00,17.0,NaN,1.0,NaN,37.00,25.0,49.0,400.0,26.00,300.00,26.00
7322,Rabbit,Indonesia,Rexsi Agrinak,14.0,14.00,16.0,NaN,7.0,8.0,NaN,NaN,NaN,2.9,49.94,2.70,55.27
8314,Cattle,Jamaica,Puerto Rican,NaN,15.00,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN


As documented above, the DAD-IS dataset contains some outliers that are clearly errors. The cell below removes them.

In [8]:
# Birth weights should not be remotely similar to adult weights.
mask = dad_is_df['Birth weight males'] > 0.9*dad_is_df['Weight males']
dad_is_df.loc[mask, 'Birth weight males'] = np.NaN
dad_is_df.loc[mask, 'Weight males'] = np.NaN

mask = dad_is_df['Birth weight females'] > 0.9*dad_is_df['Weight females']
dad_is_df.loc[mask, 'Birth weight females'] = np.NaN
dad_is_df.loc[mask, 'Weight females'] = np.NaN

# None of these animals has a litter size > 30 (being very conservative)
litter_cols = ['Litter size AVG', 'Litter size MIN', 'Litter size MAX']

# Species where litters are surely < 10 (being very conservative, see comments below).
species_l10 = 'Goat,Cattle,Sheep'.split(',')
for lc in litter_cols:
    mask = dad_is_df[lc] > 30
    dad_is_df.loc[mask, lc] = np.NaN
    
    mask = np.logical_and(dad_is_df[lc] > 10, dad_is_df.Specie.isin(species_l10))
    dad_is_df.loc[mask, lc] = np.NaN
    
# ^^ There are a few goat breeds marked as having litter sizes > 10, even > 100.
# These numbers are absurd and must be in error. 
# Data show that even quadruplets are rare, <2% of pregnancies and <3% of live young.
# These numbers do depend on the genotype, however, meaning that larger litters are selectable.
# See references on goat litter sizes: 
# 1. M. Peaker, Gestation period and litter size in the goat. Br. Vet. J. 134, 379–383 (1978).
# 2. M. Mellado, et al., Relationship between litter birthweight and litter size in five goat genotypes. Anim. Produc. Sci. 51, 490–490 (2011).

# There are also two Iraqi cattle breeds listed as producing 3.6 young/litter. These probably in error because they are very similar records and 
# an average 3.6 young/litter for cattle is well beyond typical. See the refs below, which establish typical twinning rates ≈1-5%
# with exceptional twinning rates being ≈10%. So 3.6 is an unreasonable average litter size for cattle. 
# 1. J. J. Rutledge, Twinning in cattle. J. Anim. Sci. 40, 803–815 (1975).
# 2. P. M. Fricke, Twinning in Dairy Cattle. The Professional Animal Scientist 17, 61–67 (2001).
mask = np.logical_and(dad_is_df[lc] > 3, dad_is_df.Specie == 'Cattle')
dad_is_df.loc[mask, lc] = np.NaN

# None of these animals has an interbirth interval less than 20 days (being very conservative)
# Even rabbits have a gestation period > 20 days (≈30 days)
# See W. K. Wilson, F. J. Dudley, The duration of gestation in rabbit breeds and crosses. J. Genet. 50, 384–391 (1952).
interval_cols = ['Parturition interval (day) AVG', 'Parturition interval (day) MIN', 'Parturition interval (day) MAX']
for ic in interval_cols:
    mask = dad_is_df[ic] <= 20
    dad_is_df.loc[mask, ic] = np.NaN
    
# Cattle give birth first at a few years, not > 10 (very conservatively).
# TODO: give references here. 
birth_age_cols = [
    'Age first parturition (month) AVG',
    'Age first parturition (month) MIN',
    'Age first parturition (month) MAX',] 
for ac in birth_age_cols:
    mask = np.logical_and(dad_is_df[ac] > 10*365/12.0,
                          dad_is_df.Specie == 'Cattle')
    dad_is_df.loc[mask, ac] = np.NaN


The cell below calculates derived data for the remaining species and puts units in the column names for clarity. 

In [9]:
# Calculate the number of young per year from the litter size and interval between litters.
litter_size = dad_is_df['Litter size AVG']

# Have to convert 0 interval into NaN otherwise we get infinite young per year
litter_interval = dad_is_df['Parturition interval (day) AVG']

young_per_year = litter_size / (litter_interval/365)
dad_is_df['Young per year AVG (number)'] = young_per_year

mean_adult_weight_kg = dad_is_df[['Weight males', 'Weight females']].mean(axis=1)
dad_is_df['Adult weight AVG (kg)'] = mean_adult_weight_kg
dad_is_df['Adult weight AVG (g)'] = mean_adult_weight_kg*1000

mean_birth_weight_kg = dad_is_df[['Birth weight males', 'Birth weight females']].mean(axis=1)
dad_is_df['Birth weight AVG (kg)'] = mean_birth_weight_kg
dad_is_df['Birth weight AVG (g)'] = mean_birth_weight_kg*1000

neonate_mass_kg = dad_is_df['Birth weight AVG (kg)']
young_per_year = dad_is_df['Young per year AVG (number)']
dad_is_df['Young mass per year AVG (kg)'] = neonate_mass_kg*young_per_year
dad_is_df['Young mass per year AVG (g)'] = neonate_mass_kg*young_per_year*1000

days_per_month = 365/12.0
month_cols = ['Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females']
for c in month_cols:
    day_col = c + ' (day)'
    dad_is_df[day_col] = dad_is_df[c]*days_per_month

# Provide parturition ages in days
base_colname = 'Age first parturition'
d_per_month = 365/12
for stat in 'AVG,MIN,MAX'.split(','):
    col = '{0} (month) {1}'.format(base_colname, stat)
    new_col = '{0} (day) {1}'.format(base_colname, stat)
    dad_is_df[new_col] = dad_is_df[col]*d_per_month
    
# Provide maturity ages in months
base_colname = 'Age first parturition'
d_per_month = 365/12
for stat in 'AVG,MIN,MAX'.split(','):
    col = '{0} (month) {1}'.format(base_colname, stat)
    new_col = '{0} (day) {1}'.format(base_colname, stat)
    dad_is_df[new_col] = dad_is_df[col]*d_per_month

# Put units in the relevant column names
dad_is_clean_df = dad_is_df.rename(columns=colname_map)

# Save a new cleaner file with units for the relevant columns.
dad_is_clean_df.to_csv('../data/010522_FAO_DAD-IS_cleaned.csv')
dad_is_clean_df.columns.to_list()

['Country',
 'ISO3',
 'Specie',
 'Breed/Most common name',
 'Language',
 'Description',
 'Transboundary name',
 'Other name',
 'Provision Uses',
 'Regulation and Maintenance Uses',
 'Cultural Uses',
 'Description Of Specific Uses',
 'Production environment/management systems',
 'Literature related to uses and ecosystem services',
 'Additional information',
 'Additional information comments',
 'Efabis cultural role comment',
 'Efabis cultural value',
 'Adaptability to specific environment',
 'Specific resistance or tolerance',
 'Specific reproductive characteristic',
 'Special characteristic of product',
 'Other special qualities',
 'Reference for special qualities',
 'Efabis genetic features',
 'Efabis environmental role',
 'Efabis adaptability to marginal land',
 'Body conformation',
 'Coat description',
 'Coat quality',
 'Comb type',
 'Skin colour',
 'Shank and foot colour',
 'Plumage colour',
 'Pattern within feather',
 'Avian classification',
 'Color comments',
 'Efabis main colour

In [10]:
# Save number of individual breeds for the relevant columns
counts_by_species = dad_is_clean_df.groupby('Specie')['Breed/Most common name'].count()
counts_by_species.name = 'breed_count'
counts_by_species.to_csv('../data/010522_FAO_DAD-IS_breed_counts_by_species.csv')

In [11]:
# Print the species for which there are > 100 breed representatives
mask = counts_by_species > 100
species_enough_data = counts_by_species[mask].index.tolist()
species_enough_data

['Buffalo',
 'Cattle',
 'Chicken',
 'Duck',
 'Goat',
 'Horse',
 'Pig',
 'Rabbit',
 'Sheep']

In [12]:
dad_is_clean_df.columns.tolist()

['Country',
 'ISO3',
 'Specie',
 'Breed/Most common name',
 'Language',
 'Description',
 'Transboundary name',
 'Other name',
 'Provision Uses',
 'Regulation and Maintenance Uses',
 'Cultural Uses',
 'Description Of Specific Uses',
 'Production environment/management systems',
 'Literature related to uses and ecosystem services',
 'Additional information',
 'Additional information comments',
 'Efabis cultural role comment',
 'Efabis cultural value',
 'Adaptability to specific environment',
 'Specific resistance or tolerance',
 'Specific reproductive characteristic',
 'Special characteristic of product',
 'Other special qualities',
 'Reference for special qualities',
 'Efabis genetic features',
 'Efabis environmental role',
 'Efabis adaptability to marginal land',
 'Body conformation',
 'Coat description',
 'Coat quality',
 'Comb type',
 'Skin colour',
 'Shank and foot colour',
 'Plumage colour',
 'Pattern within feather',
 'Avian classification',
 'Color comments',
 'Efabis main colour